In [1]:
%run main.py --lr 0.0001 --num_workers 4 --num_epochs 20 --balance_weights --weights_smooth 0.1

Using device: cuda
Loading training and validation datasets...
Datasets successfully loaded
Loading model...
Model loaded
Calculating class weights...
Class weights obtained
Starting training loop, Epochs: 20, Learning Rate: 0.0001
Epoch [1/20]
Epoch [1/20], Batch [1000/7500], Loss: 1.4297, Accuracy: 0.592
Epoch [1/20], Batch [2000/7500], Loss: 1.3507, Accuracy: 0.610
Epoch [1/20], Batch [3000/7500], Loss: 1.3072, Accuracy: 0.618


KeyboardInterrupt: 

In [ ]:
%run main.py --lr 0.0001 --num_workers 4 --num_epochs 20 --balance_weights --weights_smooth 0.05

In [ ]:
%run main.py --lr 0.0001 --num_workers 4 --num_epochs 20 --balance_weights --weights_smooth 0.01

In [ ]:
%run main.py --lr 0.0001 --num_workers 4 --num_epochs 20 --over_sample